In [143]:
import pandas as pd
import json

In [144]:
dfSET_PATH = '../dfsets/downsampled_50_50.csv'
EXPORTED_GENERATIONS_PATH = '../dfsets/auto_gpt4_gpt3_50_50_generations_v2.csv'
PROMPT_NAME = 'match_prompt_autogpt4_fewshot_2_eval2_temp_0_gpt3.5-0301_v2'

In [145]:
# Replace 'your_file.csv' with the actual CSV file path
df = pd.read_csv(EXPORTED_GENERATIONS_PATH)


In [146]:
# remove column "metadf"
df = df.drop(columns=['metadf'])
df = df.drop(columns=['name'])

# add column "eval"

df['eval'] = ""

df["eval"] = df["eval"].apply(lambda x: x.replace('{"completion":', "").replace('}', "").replace('"',""))
df["completion"] = df["completion"].apply(lambda x: x.replace('{"completion":', "").replace('}', "").replace('"',""))
try: 
    df["prompt"] = df["prompt"].apply(lambda x: json.loads(x)[0][0]["content"]) 
except:
    pass



In [147]:
df

,traceId,model,startTime,endTime,prompt,completion,eval
0,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:46.621Z,2023-11-01T00:47:47.346Z,"[[{""content"":""Decide if this answer correspond...","""no match""",
1,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:39.778Z,2023-11-01T00:47:46.617Z,"[[{""content"":""Q: Are the following two product...","""The first product is a refill for a lined not...",
2,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:38.564Z,2023-11-01T00:47:39.382Z,"[[{""content"":""Decide if this answer correspond...","""Match.""",
3,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:32.139Z,2023-11-01T00:47:38.560Z,"[[{""content"":""Q: Are the following two product...","""Both products are Maxxis Minion DHR II tires ...",
4,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:30.987Z,2023-11-01T00:47:31.679Z,"[[{""content"":""Decide if this answer correspond...","""match""",
...,...,...,...,...,...,...,...
437,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:16.669Z,2023-10-31T23:25:21.915Z,"[[{""content"":""Q: Are the following two product...","""The first product is a video switcher that al...",
438,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:14.380Z,2023-10-31T23:25:15.328Z,"[[{""content"":""Decide if this answer correspond...","""No match.""",
439,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:06.628Z,2023-10-31T23:25:14.377Z,"[[{""content"":""Q: Are the following two product...","""The first product is a pack of Instax Mini Ra...",
440,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:05.233Z,2023-10-31T23:25:06.111Z,"[[{""content"":""Decide if this answer correspond...","""No match.""",


In [148]:
def merge_rows(row1, row2):
    merged_row = {}
    # copy all columns from row2 and add to merged_row
    for column in row2.keys():
        merged_row[column] = row2[column]
    merged_row['eval'] = row1['completion'] 
    return merged_row


In [149]:
merged_df = []
for i in range(0, len(df), 2):
    if i + 1 < len(df):
        merged_row = merge_rows(df.iloc[i], df.iloc[i + 1])
        merged_df.append(merged_row)

merged_df = pd.dfFrame(merged_df)


In [150]:
# sort by column "startTime" in ascending order
# merged_df = merged_df.sort_values(by=['startTime'])

In [151]:
df_dfset = pd.read_csv(dfSET_PATH)


221


In [152]:
# add column "label" and "predicted_label"
merged_df['predicted_label'] = ""
merged_df['label'] = ""
merged_df['error'] = ""

# copy column "label" from df_dfset to merged_df
for i in range(len(merged_df)):
    label = df_dfset['label'][len(df_dfset)-i-1]
    merged_df['label'][i] = label
    predicted_label = df_dfset[PROMPT_NAME][len(df_dfset)-i-1]
    merged_df['predicted_label'][i] = predicted_label
    if label != predicted_label:
        if label == False:
            merged_df['error'][i] = "FP"
        elif label == True:
            merged_df['error'][i] = "FN"
        else:
            print("ERROR: " + label)
    else:
        merged_df['error'][i] = "NONE"



220
219
218
217
216
215
214
213
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [153]:
merged_df

,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error
0,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:39.778Z,2023-11-01T00:47:46.617Z,"[[{""content"":""Q: Are the following two product...","""The first product is a refill for a lined not...","""no match""",False,True,FN
1,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:32.139Z,2023-11-01T00:47:38.560Z,"[[{""content"":""Q: Are the following two product...","""Both products are Maxxis Minion DHR II tires ...","""Match.""",True,True,NONE
2,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:22.573Z,2023-11-01T00:47:30.984Z,"[[{""content"":""Q: Are the following two product...","""Both products are related to the Traveler's N...","""match""",True,True,NONE
3,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:14.334Z,2023-11-01T00:47:21.359Z,"[[{""content"":""Q: Are the following two product...","""Both products are related to Traveler's noteb...","""no match""",False,True,FN
4,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-11-01T00:47:07.459Z,2023-11-01T00:47:13.134Z,"[[{""content"":""Q: Are the following two product...","""Both products have the same model number (CAZ...","""Match.""",True,True,NONE
...,...,...,...,...,...,...,...,...,...,...
216,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:30.670Z,2023-10-31T23:25:37.407Z,"[[{""content"":""Q: Are the following two product...","""The first product is a Hikvision DVR with 16 ...","""No match.""",False,False,NONE
217,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:23.095Z,2023-10-31T23:25:29.324Z,"[[{""content"":""Q: Are the following two product...","""Both products are related to Fujifilm Instax ...","""No match.""",False,False,NONE
218,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:16.669Z,2023-10-31T23:25:21.915Z,"[[{""content"":""Q: Are the following two product...","""The first product is a video switcher that al...","""No match.""",False,False,NONE
219,a93975a0-a858-44cc-bbb2-265b51674484,gpt-3.5-turbo-0301,2023-10-31T23:25:06.628Z,2023-10-31T23:25:14.377Z,"[[{""content"":""Q: Are the following two product...","""The first product is a pack of Instax Mini Ra...","""No match.""",False,False,NONE


In [154]:
# Replace 'merged_file.csv' with the desired output file name
merged_df.to_csv(EXPORTED_GENERATIONS_PATH.replace(".csv","") + '_merged.csv', index=True)
